In [ ]:
# 필요한 라이브러리 다운로드
!pip3 install requests
!pip3 install bs4 #beautifulsoup4
!pip3 install selenium

In [ ]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
#path 변수에 자신의 chromedriver가 있는 파일 위치를 할당

#window : ~/chromedriver.exe
#mac : ~/chromedriver (.exe 빼기) 
path = 'C:/Users/sujin/Desktop/PythonProjects/crawling/chromedriver.exe'

driver = webdriver.Chrome(path)

In [ ]:
years = ['2018', '2019', '2020', '2021']
period = ['HALF001','HALF002']
names = ['','금융/재정/조세','교육/보육/가족','국방/병무','행정/안전/질서',\
        '문화/체육/관광','농림/수산/식품','환경/기상','산업/중소기업/에너지',\
        '보건/복지/고용','국토/교통']

In [ ]:
def crawling_(year, period):
    fields = []
    nums = []
    titles = []
    texts = []
    for i in range(1,11):
        driver.get('https://whatsnew.moef.go.kr/mec/ots/dif/difList.do?comBaseCd=DIFTYPCD&temp={}&temp2={}'.format(year, period))
        driver.find_element_by_css_selector('body > div.contfotbg > div > div.conttexds > div > div > ul > li:nth-child({}) > a > h3'.format(i)).click()
        time.sleep(2)
        for j in range(1,100):
            try:
                driver.find_element_by_css_selector('body > div.subcontentop > div.contfotbg > div > div.conttexds > div > table > tbody > tr:nth-child({}) > td.texalt > a'.format(j)).click()
            except:
                continue
            time.sleep(2)
            title = driver.find_element_by_css_selector('body > div.subcontentop > div.contfotbg > div > div.conttexds > div > div > div > div.con01_tit > div.con01_tit_txt.fl > p.con01_tit_p1')
            contents = driver.find_element_by_css_selector('body > div.subcontentop > div.contfotbg > div > div.conttexds > div > div > div')
            titles.append(title.text)
            fields.append(names[i])
            texts.append(contents.text)
            nums.append(j)
            driver.back()
    df = pd.DataFrame({'분야':fields,'번호':nums,'정책명':titles,'정책내용':texts})
    df.reset_index(drop=True)
    return df

In [ ]:
year = '2018'
periods = ['HALF001','HALF002']
for per in periods:
    df = crawling_(year,per)
    df.to_csv("정책_{}_{}.csv".format(year, per[-2:]), index=False)